In [6]:
from ultralytics.data.converter import convert_coco

# 转换COCO关键点为YOLOv8格式，含关键点（pose）！
convert_coco(labels_dir="../yolo", use_keypoints=True)


Annotations /home/hope/R&D_Pro/HRNet/data/bsf2023/annotations/yolo/coco_train_cat0_superclean.json:   0%|          | 0/1246 [00:00<?, ?it/s]


TypeError: must be real number, not NoneType

In [4]:
import json
import math

json_path = '../yolo/coco_train_cat0.json'
with open(json_path) as f:
    data = json.load(f)

def has_non_number(x):
    # 检查nan、None、字符串等非法数值
    if x is None:
        return True
    if isinstance(x, float) and (math.isnan(x) or math.isinf(x)):
        return True
    return not isinstance(x, (int, float))

clean_anns = []
for ann in data['annotations']:
    # 检查bbox合法
    bbox_ok = (
        'bbox' in ann and
        isinstance(ann['bbox'], list) and
        len(ann['bbox']) == 4 and
        all(not has_non_number(x) for x in ann['bbox'])
    )
    # 检查keypoints合法
    kp_ok = (
        'keypoints' in ann and
        isinstance(ann['keypoints'], list) and
        all(not has_non_number(x) for x in ann['keypoints'])
    )
    # 检查其它字段也无None
    others_ok = all(v is not None for k, v in ann.items() if k not in ['bbox', 'keypoints'])
    if bbox_ok and kp_ok and others_ok:
        clean_anns.append(ann)
    else:
        print("丢弃有问题的标注：", ann)

data['annotations'] = clean_anns
with open(json_path.replace('.json', '_superclean.json'), 'w') as f:
    json.dump(data, f)
print('清洗完毕，已保存新json！')


清洗完毕，已保存新json！
